radius, distance, 3자리, 동일한 숫자 5회 반복, 5초 지연\
conv 3layer


In [ ]:
import torch                    #ok
import torch.nn as nn             
import torch.nn.functional as F
from torch.optim import Adam
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# 신경망 정의
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # First convolutional layer
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        # Second convolutional layer
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # Third convolutional layer
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 3 * 3, 512)  # Adjust the size accordingly
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        # Apply the first convolutional layer and pooling
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        # Apply the second convolutional layer and pooling
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        # Apply the third convolutional layer and pooling
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        # Flatten the tensor
        x = x.view(-1, 128 * 3 * 3)
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

#model = Net()
#print(model)


# 모델 로드
model = Net()
PATH = 'C:\\xxpexx\\Test\\'
model.load_state_dict(torch.load(PATH + 'mnist_model.pt', map_location='cpu'))
model.eval()

def nothing(x):
    pass
cap = cv2.VideoCapture(1)

cv2.namedWindow("Trackbars")
cv2.createTrackbar("Min Radius", "Trackbars", 10, 30, nothing)
cv2.createTrackbar("Max Radius", "Trackbars", 20, 80, nothing)
cv2.createTrackbar("Distance Threshold", "Trackbars", 50, 200, nothing)

if cap.isOpened():
    while True:
        ret, img = cap.read()
        if ret:
            min_radius = cv2.getTrackbarPos("Min Radius", "Trackbars")
            max_radius = cv2.getTrackbarPos("Max Radius", "Trackbars")
            distance_threshold = cv2.getTrackbarPos("Distance Threshold", "Trackbars")
            
            g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            thr, bin_img = cv2.threshold(g_img, 40, 255, cv2.THRESH_BINARY_INV)
            contours, hierarchy = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, 
                                                   cv2.CHAIN_APPROX_SIMPLE)
            digits = []
            try:
                for contour in contours:
                    (x, y), radius = cv2.minEnclosingCircle(contour)
                    if min_radius < radius < max_radius:
                        xs, xe = int(x-radius), int(x+radius)
                        ys, ye = int(y-radius), int(y+radius)
                        cv2.rectangle(bin_img, (xs, ys), (xe, ye), (200, 0, 0), 1)
                        roi = bin_img[ys:ye, xs:xe]
                        
                        # 이미지를 28x28로 조정하고, 0에서 1 사이의 값을 갖도록 정규화
                        roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
                        roi = roi.astype("float32") / 255.0
                        roi = np.expand_dims(roi, axis=0)
                        
                        # PyTorch 텐서로 변환
                        roi_tensor = torch.from_numpy(roi).unsqueeze(0)
                        
                        # 모델을 사용하여 예측 진행
                        output = model(roi_tensor)
                        _, predicted = torch.max(output, 1)
                        num = predicted.item()
                        digits.append((num, x))
                        
                        cv2.putText(bin_img, str(num), (xs, ys), cv2.FONT_HERSHEY_PLAIN, 2, (200, 0, 0))
                digits.sort(key=lambda d: d[1])  # x 좌표에 따라 정렬
                grouped_digits = []
                group = [digits[0]]
                for i in range(1, len(digits)):
                    distance = digits[i][1] - digits[i-1][1]  # 중점 간 거리가 50 픽셀 이내일 경우
                    # print(f"Distance between digit {i-1} and {i}: {distance}")
                    if distance < 50:
                        group.append(digits[i])
                    else:
                        grouped_digits.append(group)
                        group = [digits[i]]
                grouped_digits.append(group)  # 마지막 그룹 추가
                
                # 세 자리 숫자 추출 및 출력
                for group in grouped_digits:
                    if len(group) == 3:  # 세 자리 숫자인 경우
                        three_digit_number = ''.join([str(d[0]) for d in group])
                        print("결과",three_digit_number)
                        
                        # if three_digit_number in number_counts:
                        #     number_counts[three_digit_number] += 1
                        # else:
                        #     number_counts[three_digit_number] = 1
                        # if number_counts[three_digit_number] == 3:
                        #     print(three_digit_number)
                        #     number_counts[three_digit_number] = 0
                        #     time.sleep(5)
                        
                        
                        #arduino.write(three_digit_number.encode())
            
            except Exception as e:
                pass

            cv2.imshow("Video Capture", bin_img)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print("No Frame...")
            break
else:
    print("Camera not opened...")
cap.release()
cv2.destroyAllWindows()    

In [ ]:
import torch                              #okay
import torch.nn as nn                 
import torch.nn.functional as F
from torch.optim import Adam
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import time

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  
        x = self.fc(x)
        return F.log_softmax(x)

model = Net()
PATH = 'C:\\gg\\'
model.load_state_dict(torch.load(PATH + 'model_for_RO_CNN.pt', map_location='cpu'))
model.eval()

def nothing(x):
    pass
cap = cv2.VideoCapture(1)

cv2.namedWindow("Trackbars")
cv2.createTrackbar("Min Radius", "Trackbars", 10, 30, nothing)
cv2.createTrackbar("Max Radius", "Trackbars", 20, 80, nothing)
cv2.createTrackbar("Distance Threshold", "Trackbars", 50, 200, nothing)

if cap.isOpened():
    while True:
        ret, img = cap.read()
        if ret:
            min_radius = cv2.getTrackbarPos("Min Radius", "Trackbars")
            max_radius = cv2.getTrackbarPos("Max Radius", "Trackbars")
            distance_threshold = cv2.getTrackbarPos("Distance Threshold", "Trackbars")
            
            g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            thr, bin_img = cv2.threshold(g_img, 110, 255, cv2.THRESH_BINARY_INV)
            contours, hierarchy = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, 
                                                   cv2.CHAIN_APPROX_SIMPLE)
            digits = []
            try:
                for contour in contours:
                    (x, y), radius = cv2.minEnclosingCircle(contour)
                    if min_radius < radius < max_radius:
                        xs, xe = int(x-radius), int(x+radius)
                        ys, ye = int(y-radius), int(y+radius)
                        cv2.rectangle(bin_img, (xs, ys), (xe, ye), (200, 0, 0), 1)
                        roi = bin_img[ys:ye, xs:xe]
                        
                        roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
                        roi = roi.astype("float32") / 255.0
                        roi = np.expand_dims(roi, axis=0)
                        
                        roi_tensor = torch.from_numpy(roi).unsqueeze(0)
                        
                        output = model(roi_tensor)
                        _, predicted = torch.max(output, 1)
                        num = predicted.item()
                        digits.append((num, x))
                        
                        cv2.putText(bin_img, str(num), (xs, ys), cv2.FONT_HERSHEY_PLAIN, 2, (200, 0, 0))
                
                digits.sort(key=lambda d: d[1])  
                grouped_digits = []
                group = [digits[0]]
                
                for i in range(1, len(digits)):
                    distance = digits[i][1] - digits[i-1][1]  # 중점 간 거리
                    # print(f"Distance between digit {i-1} and {i}: {distance}")
                    if distance < 50:
                        group.append(digits[i])
                    else:
                        grouped_digits.append(group)
                        group = [digits[i]]
                grouped_digits.append(group)  
                #print(grouped_digits)
                # 세 자리 숫자 추출 및 출력
                for group in grouped_digits:
                    if len(group) == 3:  
                        three_digit_number = ''.join([str(d[0]) for d in group])
                        print("결과:",three_digit_number)
                        time.sleep(5)
                    # previous_numbers.append(three_digit_number)
                    # if len(previous_numbers) > 3:
                    #     previous_numbers.pop(0)
                    # if len(previous_numbers) == 3 and previous_numbers[0] == previous_numbers[1] == previous_numbers[2]:
                    #     print("최종",three_digit_number)
                        
                        
                        
                        #arduino.write(three_digit_number.encode())
            
            except Exception as e:
                pass

            cv2.imshow("Video Capture", bin_img)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print("No Frame...")
            break
else:
    print("Camera not opened...")
cap.release()
cv2.destroyAllWindows()    

결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218
결과: 218


조금 다른 거


In [ ]:
import torch                    
import torch.nn as nn                 
import torch.nn.functional as F
from torch.optim import Adam
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# 신경망 정의
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 3 * 3, 512)  # Adjust the size accordingly
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 128 * 3 * 3)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

#model = Net()
#print(model)


model = Net()
PATH = 'C:\\xxpexx\\Test\\'
model.load_state_dict(torch.load(PATH + 'mnist_model.pt', map_location='cpu'))
model.eval()



# def preprocess_image(img):
#     """
#     Apply preprocessing to the image such as blurring and thresholding
#     to make digit extraction more robust.
#     """
#     # Blurring to reduce noise
#     img_blurred = cv2.GaussianBlur(img, (5, 5), 0)
#     # Adaptive thresholding for more robust binary image creation
#     img_thresh = cv2.adaptiveThreshold(img_blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#                                        cv2.THRESH_BINARY_INV, 11, 2)
#     return img_thresh




def nothing(x):
    pass
cap = cv2.VideoCapture(1)

cv2.namedWindow("Trackbars")
cv2.createTrackbar("Min Radius", "Trackbars", 10, 30, nothing)
cv2.createTrackbar("Max Radius", "Trackbars", 20, 80, nothing)
cv2.createTrackbar("Distance Threshold", "Trackbars", 50, 200, nothing)

if cap.isOpened():
    while True:
        ret, img = cap.read()
        if ret:
            min_radius = cv2.getTrackbarPos("Min Radius", "Trackbars")
            max_radius = cv2.getTrackbarPos("Max Radius", "Trackbars")
            distance_threshold = cv2.getTrackbarPos("Distance Threshold", "Trackbars")
            
            g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            thr, bin_img = cv2.threshold(g_img, 110, 255, cv2.THRESH_BINARY_INV)
            contours, hierarchy = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, 
                                                   cv2.CHAIN_APPROX_SIMPLE)


            # g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # bin_img = preprocess_image(g_img)
            # contours, _ = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            digits = []
            
            try:
                for contour in contours:
                    (x, y), radius = cv2.minEnclosingCircle(contour)
                    if min_radius < radius < max_radius:
                        xs, xe = int(x-radius), int(x+radius)
                        ys, ye = int(y-radius), int(y+radius)
                        cv2.rectangle(bin_img, (xs, ys), (xe, ye), (200, 0, 0), 1)
                        roi = bin_img[ys:ye, xs:xe]
                        
                        # 이미지를 28x28로 조정하고, 0에서 1 사이의 값을 갖도록 정규화
                        roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
                        roi = roi.astype("float32") / 255.0
                        roi = np.expand_dims(roi, axis=0)
                        
                        # PyTorch 텐서로 변환
                        roi_tensor = torch.from_numpy(roi).unsqueeze(0)
                        
                        # 모델을 사용하여 예측 진행
                        output = model(roi_tensor)
                        _, predicted = torch.max(output, 1)
                        num = predicted.item()
                        digits.append((num, x))
                        
                        cv2.putText(bin_img, str(num), (xs, ys), cv2.FONT_HERSHEY_PLAIN, 2, (200, 0, 0))
                
                digits.sort(key=lambda d: d[1])  # x 좌표에 따라 정렬
                grouped_digits = []
                group = [digits[0]]
                
                for i in range(1, len(digits)):
                    if digits[i][1] - digits[i-1][1] < 50:  # Distance threshold
                        group.append(digits[i])
                    else:
                        if len(group) == 3:
                            grouped_digits.append(group)
                        group = [digits[i]]
                if len(group) == 3:
                    grouped_digits.append(group)

                # Extract and validate three-digit numbers
                for group in grouped_digits:
                    three_digit_number = ''.join([str(d[0]) for d in group])
                    previous_numbers.append(three_digit_number)
                    if len(previous_numbers) > 3:
                        previous_numbers.pop(0)
                    if len(previous_numbers) == 3 and previous_numbers[0] == previous_numbers[1] == previous_numbers[2]:
                        print(three_digit_number)    
                    #print(three_digit_number)

            except Exception as e:
                pass

            cv2.imshow("Video Capture", bin_img)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print("No Frame...")
            break
else:
    print("Camera not opened...")
cap.release()
cv2.destroyAllWindows()